
# Quantum Circuit Picker, Related Works

| Neural Network Architectures | Gates              | Platform/Interface           |      Reference
| -----------------------------|:-:|------------------------------|:-:
| Artificial Neural Networks (ANN) | ROT                | pennylane                    |Data-reuploading classifier [1]
| Quantum CNN (kernel)         | RY, RandomLayers   | pennylane, torch             |Quanvolutional Neural Networks [2]
| Quantum GAN         | Hadamard, RX, RY, RZ, CNOT  | Google Cirq, tensorflow      |Quantum GAN [3]
| Photonic Quantum Neural Nets | Rotation, Squeezing, Displacement, Kerr  | Strawberryfields.fock      |Photonic Quantum Neural Nets [4]
| ANN for transfer learning | Hadamard, RY, CNOT  | pennylane, torch      | Quantum Transfer Learning [5]
| Linear solver | Hadamard, CZ, CNOT  | pennylane      | Variational Quantum Linear Solver [6]
| Coherent Linear solver | Hadamard, CZ, CNOT, CRY, RY  | pennylane      | Coherent Variational Quantum Linear Solver [7]
| Multiclass Classification | ROT, CNOT  | pennylane      | Multiclass Margin Classifier [8]
| RNN | Hadamard, CSWAP  | pennylane      | Graph RNN for Multiclass Margin Classifier [9]
| Optical Neural Network | Interferometer, Kerr, FockState  | pennylane      | Optical Neural Network [10]


* [1] https://pennylane.ai/qml/demos/tutorial_data_reuploading_classifier.html
* [2] https://github.com/PlanQK/TrainableQuantumConvolution
* [3] https://pennylane.ai/qml/demos/tutorial_QGAN.html
* [4] https://pennylane.ai/qml/demos/quantum_neural_net.html
* [5] https://pennylane.ai/qml/demos/tutorial_quantum_transfer_learning.html
* [6] https://pennylane.ai/qml/demos/tutorial_vqls.html
* [7] https://pennylane.ai/qml/demos/tutorial_coherent_vqls.html
* [8] https://pennylane.ai/qml/demos/tutorial_multiclass_classification.html
* [9] https://pennylane.ai/qml/demos/qgrnn.html
* [10] https://pennylane.ai/qml/demos/qonn.html



We consider the following factors to pick quantum circuits

| nntyp | interface              | simulator
| :-:|:-:|:-:
| ann, cn, gan, gnn, anntf, rnn | torch                | "default.qubit", "cirq.simulator"


In [9]:
import sys
import pennylane as qml
import numpy as np

In [68]:
def qcircuit_picker(nntype, params, features, wires, simulator=None, interface=None, seed=None):
    
    if simulator is None: 
        simulator = "default.qubit"
    if interface is None:
        interface = "torch"
        
    if nntype == "ann":
        return AnnLayer(params, features, wires)
    elif nntype == "cnn":
        return CnnLayer(params, features, wires)
    elif nntype == "anntf":
        return AnnTfLayer()
    elif nntype == "gan":
        return "GAN quantum circuit picker under development..."
    elif nntype == "gnn":
        return "GNN quantum circuit picker under development..."
#     switcher = {
#         0: ann(),
#         1: cnn(),
#         2: anntf(),
#         3: gan(),
#         4: gnn()
#     }
#     return switcher.get(argument, "invalid neural net type")


In [54]:
# a layer in an ANN
# This layer returns multiple measurements per wire
# For each wire, a ROT gate embedds the feature (i.e. the data sample of length 3) followed by
# another ROT gate that takes the params (i.e. the weights to optimize) as input
# The features could be the output of a preceeding layer (classical or quantum)

def AnnLayer(params, features, wires):
            
        """A variational quantum circuit representing the Universal classifier.

        Args:
            params (array[float]): array of parameters
            features (array[float]): single input vector

        Returns:
            float: measurements per wire
        """
        i = 0
        ret = []
        for p in params:
            qml.Rot(*features, wires=wires[i])
            qml.Rot(*p, wires=wires[i])
            i += 1
            ret.append(qml.expval(qml.PauliZ(i)))
        return ret

In [55]:
# test case per layer
num_wires = 5
params = np.pi * np.random.random_sample((num_wires, 3))
features = np.random.uniform(0, np.pi, 3)
measurements =  AnnLayer(params, features, range(num_wires))
print(measurements)

[expval(PauliZ(wires=[1])), expval(PauliZ(wires=[2])), expval(PauliZ(wires=[3])), expval(PauliZ(wires=[4])), expval(PauliZ(wires=[5]))]


In [61]:
# test case using qcircuit_picker
num_wires = 5
params = np.pi * np.random.random_sample((num_wires, 3))
features = np.random.uniform(0, np.pi, 3)
measurements = qcircuit_picker("ann", params, features, range(num_wires))
print(measurements)

[expval(PauliZ(wires=[1])), expval(PauliZ(wires=[2])), expval(PauliZ(wires=[3])), expval(PauliZ(wires=[4])), expval(PauliZ(wires=[5]))]


In [69]:
# build a N-dimensional kernel for CNN, N = 2, 3
def CnnLayer(params, features, wires, seed=None):
#     print("under development...")
#     return "cnnlayer"
    if seed is None:
            seed = np.random.randint(low=0, high=10e6)
    flist = list(features)
    plist = list(params)
    # Encoding of features (i.e. classical input values)
    ret = []
    for w in wires:
        qml.RY(flist.pop(), wires=w)
        qml.RY(plist.pop(), wires=w)
        ret.append(qml.expval(qml.PauliZ(w)))
        
    #RandomLayers(weights, wires=list(wires), seed=seed)    
    return ret
            

In [71]:
# test case per layer
num_wires = 4
#params = np.pi * np.random.random_sample((2, 2))
params = np.random.uniform(0, np.pi, num_wires)
features = np.random.uniform(0, np.pi, num_wires)
measurements =  CnnLayer(params, features, range(num_wires))
print(measurements)

[expval(PauliZ(wires=[0])), expval(PauliZ(wires=[1])), expval(PauliZ(wires=[2])), expval(PauliZ(wires=[3]))]


In [72]:
# test case using qcircuit_picker
num_wires = 4
params = np.random.uniform(0, np.pi, num_wires)
features = np.random.uniform(0, np.pi, num_wires)
measurements = qcircuit_picker("cnn", params, features, range(num_wires))
print(measurements)

[expval(PauliZ(wires=[0])), expval(PauliZ(wires=[1])), expval(PauliZ(wires=[2])), expval(PauliZ(wires=[3]))]


In [64]:
# ref example: https://pennylane.readthedocs.io/en/latest/_modules/pennylane/qnn/keras.html#KerasLayer
# ref: https://pennylane.ai/qml/demos/tutorial_quantum_transfer_learning.html
def AnnTfLayer():
    print("under development...")
    return "anntflayer"
#     for idx in range(nqubits):
#          qml.Hadamard(wires=idx)
#     for idx, element in enumerate(w):
#             qml.RY(element, wires=idx)
#     for i in range(0, nqubits - 1, 2): 
#             qml.CNOT(wires=[i, i + 1])
#     for i in range(1, nqubits - 1, 2):  
#             qml.CNOT(wires=[i, i + 1])      
#     exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
#          return tuple(exp_vals)
    

In [65]:
# test case per layer
num_wires = 5
params = np.pi * np.random.random_sample((num_wires, 3))
features = np.random.uniform(0, np.pi, 3)
anntflayer_out = qcircuit_picker("anntf", params, features, range(num_wires))
print(anntflayer_out)

under development...
anntflayer
